In [ ]:
%load_ext autoreload
%autoreload 2

import os, pathlib, shutil, google.colab
mount_path  = pathlib.Path('/content/drive/')
root_path   = mount_path / 'MyDrive/gerrymandering/2022-12'
config_path = root_path / 'config'
google.colab.drive.mount(str(mount_path))
root_path.mkdir(parents=True, exist_ok=True)
os.chdir(root_path)
shutil.rmtree(config_path, ignore_errors=True)
os.system('git clone https://github.com/drscook/config.git')

from config.imports import *
import config.utils as ut

bq = ut.BQ('redistricting')

gitcred = ut.jsonify(root_path / 'gitcred.json')
gitcred['root'] = root_path
config_repo = ut.Github(repo='config', **gitcred)
repo = ut.Github(repo='voting_predictor', **gitcred)
config_repo.sync()
repo.sync()

In [8]:
repo.sync()

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean

